In [5]:
import json
from collections import defaultdict
input_jsonl_file = 'result/20250909_091108_gaia_20250905.jsonl' 

# ----------------------------------------------------
# 步骤二：我们准备好的成本计算函数
# ----------------------------------------------------
def calculate_cost(prompt_tokens, completion_tokens):
    """
    根据给定的token数和价目表计算成本。
    价目表单位为美元/百万令牌。
    """
    # 价格单位: $/1M tokens
    PRICE_INPUT_UNDER_200K = 3.0
    PRICE_OUTPUT_UNDER_200K = 15.0
    PRICE_INPUT_OVER_200K = 6.0
    PRICE_OUTPUT_OVER_200K = 22.50

    if prompt_tokens <= 200000:
        input_price = PRICE_INPUT_UNDER_200K
        output_price = PRICE_OUTPUT_UNDER_200K
    else:
        input_price = PRICE_INPUT_OVER_200K
        output_price = PRICE_OUTPUT_OVER_200K

    cost_input = (prompt_tokens / 1_000_000) * input_price
    cost_output = (completion_tokens / 1_000_000) * output_price
    
    total_cost = cost_input + cost_output
    return total_cost

# ----------------------------------------------------
# 步骤三：主逻辑
# ----------------------------------------------------

# 1. 读取数据 (和你提供的一样)
try:
    with open(input_jsonl_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        data = [json.loads(line) for line in lines]
except FileNotFoundError:
    print(f"错误：找不到文件 '{input_jsonl_file}'")
    data = []

# 2. 初始化用于统计的数据结构
# defaultdict(list) 创建一个字典，当访问一个不存在的键时，会自动为其创建一个空列表
level_durations = defaultdict(list)
level_costs = defaultdict(list)

# 3. 遍历数据，提取信息并计算
for item in data:
    try:
        level = item["_orig"]["Level"]
        
        info = item["session"]["info"]
        duration = info["duration"]
        
        all_stats = info["call_stat"]["__ALL__"]
        prompt_tokens = all_stats["prompt_tokens"]
        completion_tokens = all_stats["completion_tokens"]
        
        # 调用函数计算成本
        cost = calculate_cost(prompt_tokens, completion_tokens)
        
        # 将数据存入对应Level的列表中
        level_durations[level].append(duration)
        level_costs[level].append(cost)
        
    except KeyError as e:
        print(f"警告：处理某条记录时缺少关键字段 {e}，已跳过。")
        continue

# 4. 计算并展示结果
print("\n" + "="*20 + " 性能分析报告 " + "="*20)

if not level_durations:
    print("没有处理任何有效数据。")
else:
    # --- 分级统计 ---
    print("\n分级平均统计:")
    # 对Level进行排序，确保输出顺序一致
    sorted_levels = sorted(level_durations.keys())
    
    for level in sorted_levels:
        durations = level_durations[level]
        costs = level_costs[level]
        
        # 计算平均值
        avg_duration = sum(durations) / len(durations) if durations else 0
        avg_cost = sum(costs) / len(costs) if costs else 0
        
        # 获取该级别的任务数量
        count = len(durations)
        
        print(f"  - {level} (共 {count} 条):")
        print(f"    - 平均时长 (Avg Duration): {avg_duration:.2f}s")
        print(f"    - 平均成本 (Avg Cost)   : ${avg_cost:.2f}")

    # --- 总体统计 ---
    print("\n总体平均统计:")
    # 将所有Level的数据合并到一起
    all_durations = [d for sublist in level_durations.values() for d in sublist]
    all_costs = [c for sublist in level_costs.values() for c in sublist]
    
    total_count = len(all_durations)
    
    # 计算总平均值
    overall_avg_duration = sum(all_durations) / total_count if total_count > 0 else 0
    overall_avg_cost = sum(all_costs) / total_count if total_count > 0 else 0
    
    print(f"  - 所有级别 (共 {total_count} 条):")
    print(f"    - 平均时长 (Avg Duration): {overall_avg_duration:.2f}s")
    print(f"    - 平均成本 (Avg Cost)   : ${overall_avg_cost:.2f}")

print("="*54 + "\n")


==================== 性能分析报告 ====================

分级平均统计:
  - 1 (共 53 条):
    - 平均时长 (Avg Duration): 463.82s
    - 平均成本 (Avg Cost)   : $1.82
  - 2 (共 86 条):
    - 平均时长 (Avg Duration): 1061.05s
    - 平均成本 (Avg Cost)   : $2.24
  - 3 (共 26 条):
    - 平均时长 (Avg Duration): 1795.08s
    - 平均成本 (Avg Cost)   : $2.77

总体平均统计:
  - 所有级别 (共 165 条):
    - 平均时长 (Avg Duration): 984.88s
    - 平均成本 (Avg Cost)   : $2.19



In [6]:
print(f"all duration: {sum(all_durations)/3600:.2f} H")
print(f"all cost: ${sum(all_costs):.4f}")

all duration: 45.14 H
all cost: $360.7413
